In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
# read the train and test dataset
import sklearn
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import svm

In [2]:
data = pd.read_csv('CaseStudy/train.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
print(data.shape)

(891, 12)


In [5]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [7]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
data.Name.tail()

886                       Montvila, Rev. Juozas
887                Graham, Miss. Margaret Edith
888    Johnston, Miss. Catherine Helen "Carrie"
889                       Behr, Mr. Karl Howell
890                         Dooley, Mr. Patrick
Name: Name, dtype: object

In [9]:
data['Title'] = data.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))

In [10]:
data['Title'] = data['Title'].replace({'Mlle':'Miss', 'Mme':'Mrs', 'Ms':'Miss'})
data['Title'] = data['Title'].replace(['Don', 'Dona', 'Rev', 'Dr','Major', 'Lady', 'Sir', 'Col', 'Capt', 'Countess', 'Jonkheer'],'Special')

In [11]:
data['Has_Cabin'] = ~data.Cabin.isnull()

In [12]:
data.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Has_Cabin
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S,Special,False
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S,Miss,True
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,Miss,False
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,Mr,True
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,Mr,False


In [13]:
data['Age'] = data.Age.fillna(data.Age.median())
data['Fare'] = data.Fare.fillna(data.Fare.median())
data['Embarked'] = data['Embarked'].fillna('S')

In [14]:
data.drop(['Cabin', 'Name', 'PassengerId', 'Ticket'], axis=1, inplace=True)

In [15]:
data['CatAge'] = pd.qcut(data.Age, q=4, labels=False )
data['CatFare']= pd.qcut(data.Fare, q=4, labels=False)

In [16]:
data = data.drop(['Age', 'Fare'], axis=1)

In [17]:
data['Fam_Size'] = data.Parch + data.SibSp

In [18]:
data['Sex'] = data['Sex'].replace({'female':0, 'male':1})
data['Embarked'] = data['Embarked'].replace({'S':0, 'C':1, 'Q':2})

In [19]:
data = data.drop(['Title'], axis=1)

In [20]:
test_data, train_data = sklearn.model_selection.train_test_split(data, train_size=.80, test_size=.20)
print(test_data.head())
print(train_data.head())

     Survived  Pclass  Sex  SibSp  Parch  Embarked  Has_Cabin  CatAge  \
687         0       3    1      0      0         0      False       0   
801         1       2    0      1      1         0      False       2   
749         0       3    1      0      0         2      False       2   
154         0       3    1      0      0         0      False       1   
797         1       3    0      0      0         0      False       2   

     CatFare  Fam_Size  
687        1         0  
801        2         2  
749        0         0  
154        0         0  
797        1         0  
     Survived  Pclass  Sex  SibSp  Parch  Embarked  Has_Cabin  CatAge  \
575         0       3    1      0      0         0      False       0   
439         0       2    1      0      0         0      False       2   
55          1       1    1      0      0         0       True       1   
756         0       3    1      0      0         0      False       1   
208         1       3    0      0      0      

In [21]:
train_x = train_data.drop(columns=['Survived'],axis=1)
train_y = train_data['Survived']

In [22]:
test_x = test_data.drop(columns=['Survived'],axis=1)
test_y = test_data['Survived']

In [23]:
model = LogisticRegression()

In [24]:
model.fit(train_x,train_y)

LogisticRegression()

In [25]:
print('Coefficient of model :', model.coef_)

Coefficient of model : [[-1.26982081 -2.7958927  -0.25565803  0.23397428  0.59542402  0.1838062
   0.2084593  -0.32445584 -0.02168375]]


In [26]:
print('Intercept of model',model.intercept_)

Intercept of model [3.89334172]


In [27]:
predict_train = model.predict(train_x)
print('Target on train data',predict_train) 

Target on train data [0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0
 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 1
 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1]


In [28]:
accuracy_train = accuracy_score(train_y,predict_train)
print('accuracy_score on train dataset : ', accuracy_train)

accuracy_score on train dataset :  0.8491620111731844


In [29]:
predict_test = model.predict(test_x)
print('Target on test data',predict_test)

Target on test data [0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0
 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 0
 0 1 1 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 1
 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0
 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0
 0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0
 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0
 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0
 1 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1
 0 0 

In [30]:
accuracy_test = accuracy_score(test_y,predict_test)
print('accuracy_score on test dataset : ', accuracy_test)

accuracy_score on test dataset :  0.7752808988764045


In [31]:
result=metrics.confusion_matrix(test_y, predict_test)
print('Confusion Matrix  on test dataset : ', result)

Confusion Matrix  on test dataset :  [[383  59]
 [101 169]]


In [32]:
X = data.drop(columns=['Survived'],axis=1)
y = data['Survived']
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=15)
print(scores, scores.mean())

[0.81666667 0.78333333 0.81666667 0.8        0.73333333 0.88333333
 0.77966102 0.77966102 0.76271186 0.79661017 0.72881356 0.76271186
 0.77966102 0.77966102 0.79661017] 0.7866290018832393


In [33]:
# now to do 
# create a column sex_male and fill wit 0 or 1
#data.insert(2, "sex_male", [21, 23, 24, 21], True)
#drop sex
#change the Embarked column to a numeric value as well
data.info

<bound method DataFrame.info of      Survived  Pclass  Sex  SibSp  Parch  Embarked  Has_Cabin  CatAge  \
0           0       3    1      1      0         0      False       0   
1           1       1    0      1      0         1       True       3   
2           1       3    0      0      0         0      False       1   
3           1       1    0      1      0         0       True       2   
4           0       3    1      0      0         0      False       2   
..        ...     ...  ...    ...    ...       ...        ...     ...   
886         0       2    1      0      0         0      False       1   
887         1       1    0      0      0         0       True       0   
888         0       3    0      1      2         0      False       1   
889         1       1    1      0      0         1       True       1   
890         0       3    1      0      0         2      False       2   

     CatFare  Fam_Size  
0          0         1  
1          3         1  
2          1    

In [34]:
data.apply(lambda row: row.Sex, axis=1)
#data.insert(2, "sex_male", )

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Length: 891, dtype: int64

In [35]:
data['sex_male'] = data.apply(lambda row: row.Sex, axis=1)

In [36]:
data.info

<bound method DataFrame.info of      Survived  Pclass  Sex  SibSp  Parch  Embarked  Has_Cabin  CatAge  \
0           0       3    1      1      0         0      False       0   
1           1       1    0      1      0         1       True       3   
2           1       3    0      0      0         0      False       1   
3           1       1    0      1      0         0       True       2   
4           0       3    1      0      0         0      False       2   
..        ...     ...  ...    ...    ...       ...        ...     ...   
886         0       2    1      0      0         0      False       1   
887         1       1    0      0      0         0       True       0   
888         0       3    0      1      2         0      False       1   
889         1       1    1      0      0         1       True       1   
890         0       3    1      0      0         2      False       2   

     CatFare  Fam_Size  sex_male  
0          0         1         1  
1          3         